In [52]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


In [53]:
bc = datasets.load_breast_cancer()
X, y = bc.data, bc.target

n_samples, n_features = X.shape

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [54]:
sc = StandardScaler() ## Mostly used in case of logistic regression, since we would be using sigmoid function

X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32)).reshape(-1,1)
y_test = torch.from_numpy(y_test.astype(np.float32)).reshape(-1,1)

In [55]:
class LogisticRegression(nn.Module):
    def __init__(self, n_input_features):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(n_input_features, 1)
        
    def forward(self, x):
        y_predicted = torch.sigmoid(self.linear(x))
        return y_predicted
    
model = LogisticRegression(n_features)   

In [56]:
learning_rate = 0.03
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [57]:
epochs = 100
for epoch in range(epochs):
    #Forward pass and loss
    y_predicted = model(X_train)
    loss = criterion(y_predicted, y_train)
    #Backward pass
    loss.backward()
    #Update and reset
    optimizer.step()
    optimizer.zero_grad()
    
    #Prompt
    if epoch%10==0:
        print(f'For epoch {epoch+1}, loss is {loss.item():.8f}')
    
with torch.no_grad():
    y_predicted = model(X_test)
    y_predicted_cls = y_predicted.round()
    acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(f'Accuracy = {acc:.4f}')

For epoch 1, loss is 0.58762997
For epoch 11, loss is 0.36270344
For epoch 21, loss is 0.28583804
For epoch 31, loss is 0.24571629
For epoch 41, loss is 0.22028738
For epoch 51, loss is 0.20235942
For epoch 61, loss is 0.18885821
For epoch 71, loss is 0.17822836
For epoch 81, loss is 0.16958684
For epoch 91, loss is 0.16238920
Accuracy = 0.9825
